# Data Parsing COCA

## coca-samples-text

In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

import re
import os

[nltk_data] Downloading package stopwords to /home/divya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Parsing Functions

In [2]:
class Article: 
    def __init__(self, number, text, sentence_texts, sentence_tokens=[], sentence_tokens_wo_sw=[]): 
        self.number = number
        self.text = text
        self.sentence_texts = sentence_texts
        self.sentence_tokens = sentence_tokens
        self.sentence_tokens_wo_sw = sentence_tokens_wo_sw
    
    def __str__(self):
        return str({self.number : {"text" : self.text[:50], "sentences" : [s[:10] for s in self.sentence_texts[:3]]}})

class File: 
    def __init__(self, filename, articles): 
        self.filename = filename
        self.articles = articles
    
    def __str__(self): 
        return str({self.filename : [str(a) for a in self.articles[:3]]})

In [3]:
def parse_articles(file_text): 
    article_texts = file_text.split("\n")

    articles = []
    for article_text in article_texts: 
        if len(article_text) == 0: 
            continue
        
        pattern = r'@@\d+ '
        rv = re.findall(pattern, article_text[:20])
        if len(rv) == 0: 
            continue
        article_number = int(rv[0][2:-1])

        pattern = r" [\.|\?|\!] "
        sentence_texts = re.split(pattern, article_text)[1:]
        if len(sentence_texts) == 0: 
            continue
        
        # article = Article(article_number, article_text, sentence_texts, sentence_tokens, sentence_tokens_wo_sw)
        article = Article(article_number, article_text, sentence_texts)
        articles.append(article)
    
    if len(articles) == 0: 
        return None
    return articles

In [4]:
def parse_filename(directory, filename): 
    file = open(directory + filename, "r", encoding="ISO-8859-1")
    file_text = file.read()
    file.close()
    
    articles = parse_articles(file_text)
    if articles == None: 
        return None
    file = File(filename, articles)
    return file

In [5]:
def get_files(directory): 
    files = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        
        file = parse_filename(directory, filename)
        if file == None: 
            continue
        files.append(file)
    return files

In [6]:
text_directory = "data/coca-samples-text/"
text_files = get_files(text_directory)
text_spok_file = [file for file in text_files if "text_spok" in file.filename][0]
text_tvm_file = [file for file in text_files if "text_tvm" in file.filename][0]

In [7]:
print(f"Number of Files: {len(text_files)}")
print(text_spok_file)

Number of Files: 8
{'text_spok.txt': ["{17141: {'text': '@@17141 ERIC @!BURNS , FOX NEWS HOST : On this wee', 'sentences': ['The media ', 'How did th', 'How did au']}}", "{21741: {'text': '@@21741 qwq @ ! DOUGLAS-FORD-ARSO : I set the fire', 'sentences': ['DOUGLAS-FO', 'I burned i', 'I cant say']}}", "{207541: {'text': '@@207541 ( BEGIN VIDEOTAPE ) HOWARD KURTZ , HOST (', 'sentences': ['Were ABC a', 'Should Geo', 'With Iraq ']}}"]}


### Format Classifier

In [8]:
import json
import nltk
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from scipy import sparse
from collections import Counter
import operator

In [9]:
def build_data(file1, file2): 
    X, Y = [], []
    for a in file1.articles: 
        X.append(a.sentence_texts)
        Y.append(0)
    for a in file2.articles: 
        X.append(a.sentence_texts)
        Y.append(1)
    return (X, Y)

In [10]:
X, Y = build_data(text_spok_file, text_tvm_file)
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=0)

### Build Classifier

In [11]:
def print_weights(clf, vocab, n=10):
    weights=clf.coef_[0]
    reverse_vocab=[None]*len(weights)
    for k in vocab:
        reverse_vocab[vocab[k]]=k

    for feature, weight in sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))[:n]:
        print("%.3f\t%s" % (weight, feature))

    print()

    for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
        print("%.3f\t%s" % (weight, feature))

In [12]:
def build_features(dataX, feature_functions):
    
    """ This function featurizes the data according to the list of parameter feature_functions """
    
    data=[]
    for tokens in dataX:
        feats={}
        
        for function in feature_functions:
            feats.update(function(tokens))

        data.append(feats)
    return data

In [13]:
def features_to_ids(data, feature_vocab):
    
    """ 
    
    This helper function converts a dictionary of feature names to a sparse representation
 that we can fit in a scikit-learn model.  This is important because almost all feature 
 values will be 0 for most documents (note: why?), and we don't want to save them all in 
 memory.

    """
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [14]:
def create_vocab(data, top_n=None):
    
    """ 
    
    This helper function converts a dictionary of feature names to unique numerical ids. 
    top_n limits the features to only the n most frequent features observed in the training data 
    (in terms of the number of documents that contains it).
    
    """
    
    counts=Counter()
    for doc in data:
        for feat in doc:
            counts[feat]+=1

    feature_vocab={}

    for idx, (k, v) in enumerate(counts.most_common(top_n)):
        feature_vocab[k]=idx
                
    return feature_vocab

In [15]:
def pipeline(trainX, devX, trainY, devY, feature_functions):

    """ This function evaluates a list of feature functions on the training/dev data arguments """
    
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data.
    feature_vocab=create_vocab(trainX_feat, top_n=100000)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    clf = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000)
    clf.fit(trainX_ids, trainY)
    print("Accuracy: %.3f" % clf.score(devX_ids, devY))
    
    return clf, feature_vocab

In [16]:
hedging_file = open("data/hedging_data.txt")
hedging_text = hedging_file.read()
hedging_file.close()
hedging_lst = [h for h in hedging_text.split("\n") if ("%" not in h and len(h) > 2)]
print(hedging_lst)

['largely', 'generally', 'often', 'rarely', 'sometimes', 'frequently', 'occasionally', 'seldom', 'usually', 'most', 'several', 'some', 'almost', 'practically', 'apparently', 'virtually', 'basically', 'approximately', 'roughly', 'somewhat', 'somehow', 'partially', 'actually', 'like', 'something', 'someone', 'somebody', 'somewhere', 'think', 'thinks', 'thought', 'believe', 'believed', 'believes', 'consider', 'considers', 'considered', 'assume', 'assumes', 'assumed', 'understand', 'understands', 'understood', 'find', 'found', 'finds', 'appear', 'appears', 'appeared', 'seem', 'seems', 'seemed', 'suppose', 'supposes', 'supposed', 'guess', 'guesses', 'guessed', 'estimate', 'estimates', 'estimated', 'speculate', 'speculates', 'speculated', 'suggest', 'suggests', 'suggested', 'may', 'could', 'should', 'might', 'surely', 'probably', 'likely', 'maybe', 'perhaps', 'unsure', 'probable', 'unlikely', 'possibly', 'possible', 'read', 'say', 'says', 'looks like', 'look like', "don't know", 'necessarily

In [17]:
def hedging_feature(sentence_texts): 
    feats = {}
    for sentence in sentence_texts: 
        for hedge in hedging_lst: 
            if hedge in sentence: 
                feats[hedge] = feats.get(hedge, 0) + 1
    return feats

In [18]:
features = [hedging_feature]
clf, vocab = pipeline(trainX, testX, trainY, testY, features)

Accuracy: 0.910


In [19]:
print_weights(clf, vocab, n=10)

-7.702	several
-5.650	allege
-5.650	alleged
-5.425	so far
-4.777	possibility
-4.466	sort of
-4.343	somewhat
-3.899	necessarily
-3.813	estimates
-3.608	couple

5.305	look like
4.075	practically
4.073	consistent with
4.022	finds
3.766	looks like
3.014	a bit
2.920	assumption
2.832	rare
2.744	appeared
2.696	few
